#### Multivariate Analysis

#### For explanation to this notebook look into the images directory, knn_imputer.jpg

In [48]:
import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import train_test_split
from sklearn.impute import MissingIndicator,SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer,SimpleImputer

In [49]:
df = pl.read_csv("Data_set/train_1.csv", columns=["Age", "Fare", "Survived"])
df.head()

Survived,Age,Fare
i64,f64,f64
0,22.0,7.25
1,38.0,71.2833
1,26.0,7.925
1,35.0,53.1
0,35.0,8.05


In [50]:
df.select([
    pl.col(col).is_null().mean().alias(col) * 100
    for col in df.columns
])

Survived,Age,Fare
f64,f64,f64
0.0,19.86532,0.0


In [51]:
X = df.drop("Survived")
y = df.select("Survived")

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()


In [53]:
for i in range(1, 11):
    knn = KNNImputer(n_neighbors=i, weights='distance')

    X_train_trf    = knn.fit_transform(X_train)
    X_test_trf     = knn.transform(X_test)
    
    lr = LogisticRegression()
    lr.fit(X_train_trf,y_train)
    y_pred = lr.predict(X_test_trf)

    print(i,'->', accuracy_score(y_test,y_pred))

1 -> 0.6033519553072626
2 -> 0.6089385474860335
3 -> 0.6033519553072626
4 -> 0.6145251396648045
5 -> 0.6089385474860335
6 -> 0.6145251396648045
7 -> 0.6145251396648045
8 -> 0.6145251396648045
9 -> 0.6089385474860335
10 -> 0.6089385474860335


In [54]:
knn = KNNImputer(n_neighbors=4,weights='distance')

X_train_trf    = knn.fit_transform(X_train)
X_test_trf     = knn.transform(X_test)

In [55]:
lr = LogisticRegression()
lr.fit(X_train_trf,y_train)
y_pred = lr.predict(X_test_trf)

accuracy_score(y_test,y_pred)

0.6145251396648045

In [56]:
si = SimpleImputer()

X_train_trf2  = si.fit_transform(X_train)
X_test_trf2   = si.transform(X_test)

In [57]:
lr = LogisticRegression()
lr.fit(X_train_trf2,y_train)
y_pred2 = lr.predict(X_test_trf2)

accuracy_score(y_test,y_pred2)

0.6145251396648045